In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import special
import h5py
from astropy.wcs import WCS
import astropy.coordinates as coord
import astropy.units as u
from pyutils import *
import types
import numpy.ma as ma
import sys

#ROOT_FOLDER = "/Volumes/Seagate Backup Plus Drive/galaxy-groups-data/"
ROOT_FOLDER = "D:\\galaxy-groups-data\\"


## Basic read-in of HDF5 data from MXXL


In [ ]:
DATA_CUT_INDEX = 1000000

In [ ]:
weights = h5py.File(ROOT_FOLDER + 'weights_3pass.hdf5', 'r')
print(list(weights))
print(list(weights['Data']))
print(list(weights['Weight']))


In [ ]:
assigned = weights['Weight/bitweight0'][0:DATA_CUT_INDEX] & 1 # choose 1 of the 2048 fiber assignment realizations with this bitstring
print(np.sum(assigned == 1), "galaxies were assigned a fiber")
print(np.sum(assigned == 0), "galaxies were NOT assigned a fiber")


# Test of nearest neighbors implementations


In [ ]:

input = weights
dec = input['Data/dec'][0:DATA_CUT_INDEX]
ra = input['Data/ra'][0:DATA_CUT_INDEX]
z_obs = input['Data/z_obs'][0:DATA_CUT_INDEX]
app_mag = input['Data/app_mag'][0:DATA_CUT_INDEX]

bright_filter = app_mag < 19.5 # makes a filter array (True/False values)
redshift_filter = z_obs > 0 # makes a filter array (True/False values)
keep = np.all([bright_filter, redshift_filter], axis=0)
dec = dec[keep]
ra = ra[keep]
z_obs = z_obs[keep]

count = len(dec)
print(count, "galaxies in HDF5 file")

# choose 1 of the 2048 fiber assignment realizations with this bitstring
fiber_assigned_0 = assigned.astype(bool)
fiber_assigned_0 = fiber_assigned_0[keep]
fiber_not_assigned_0 = np.invert(fiber_assigned_0)
indexes_not_assigned = np.argwhere(fiber_not_assigned_0)
print(np.sum(fiber_assigned_0), "galaxies were assigned a fiber")

In [ ]:
# Astropy NN Search with kdtrees
catalog = coord.SkyCoord(ra=ra[fiber_assigned_0]*u.degree, dec=dec[fiber_assigned_0]*u.degree, frame='icrs')
z_cat = z_obs[fiber_assigned_0]
to_match = coord.SkyCoord(ra=ra[fiber_not_assigned_0]*u.degree, dec=dec[fiber_not_assigned_0]*u.degree, frame='icrs')

idx, d2d, d3d = coord.match_coordinates_sky(to_match, catalog, storekdtree=False)

z_eff_a = np.copy(z_obs)
z_err_a = np.zeros(len(z_obs))

# i is the index of the full sized array that needed a NN z value
# j is the index along the to_match list corresponding to that
# idx are the indexes of the NN from the catalog
assert len(indexes_not_assigned) == len(idx)

j = 0
for i in indexes_not_assigned:
    new_z = z_cat[idx[j]]
    z_err_a[i] = abs(z_eff_a[i] - new_z) / z_eff_a[i]
    z_eff_a[i] = new_z
    j = j + 1


In [ ]:
# Examine fractional error in assigned redshifts 
plt.hist(np.log10(z_err_a[fiber_not_assigned_0]), bins=50)
plt.yscale('log')
plt.xlabel('log(fractional error) from known simulation value')
plt.ylabel('Count')
print(np.sum(z_err_a))
print(np.count_nonzero(z_err_a))


In [ ]:
# My naive NN

nn = NearestNeighbor(ra[fiber_assigned_0], dec[fiber_assigned_0], z_obs[fiber_assigned_0])

# z_eff: same as z_obs if a fiber was assigned and thus a real redshift measurement was made
# otherwise, it is an assigned value.
# nearest neighbor will find the nearest (measured) galaxy and use its redshift.
z_eff = np.copy(z_obs)
z_err = np.zeros(len(z_obs))
for i in range(0, count):
    if not fiber_assigned_0[i]:
        new_z = nn.get_z(coord.Angle(ra[i]*u.degree).radian, coord.Angle(dec[i]*u.degree).radian)        
        z_err[i] = abs(z_eff[i] - new_z) / z_eff[i]
        z_eff[i] = new_z

In [ ]:
# Examine fractional error in assigned redshifts 
plt.hist(np.log10(z_err[fiber_not_assigned_0]), bins=50)
plt.yscale('log')
plt.xlabel('log(fractional error) from known simulation value')
plt.ylabel('Count')
print(np.sum(z_err))
print(np.count_nonzero(z_err))

In [ ]:
# First ensure the non-assigned stuff is exactly the same
print("Are all galaxies that should just use z_obs the same?", np.all(np.isclose(z_eff[fiber_assigned_0],z_eff_a[fiber_assigned_0],rtol=1e-05,atol=0.0)))
print("Do all galaxies that had nearest neighbors assigned match up (me vs astropy)?", np.all(np.isclose(z_eff[fiber_not_assigned_0],z_eff_a[fiber_not_assigned_0],rtol=1e-05,atol=0.0)))

print("  Fraction of NN with same value (want 1.0):", np.sum(np.isclose(z_eff[fiber_not_assigned_0],z_eff_a[fiber_not_assigned_0],rtol=1e-05,atol=0.0)) / len(z_eff[fiber_not_assigned_0]))

# TODO why do astropy and I result in a different assigned redshift ~25% of the time


In [ ]:
ind_different = np.argwhere(np.invert(np.isclose(z_eff,z_eff_a,rtol=1e-05,atol=0.0)))
print(len(ind_different), "galaxies don't agree between implementations")

print("MINE         ASTROPY      SIMULATION")
for i in range(0, min(len(ind_different), 20)):
    print(z_eff[ind_different[i]], z_eff_a[ind_different[i]], z_obs[ind_different[i]])

## Experiment with masking arrays 


In [ ]:
all_z_obs = weights['Data/z_obs'][:]

print("Index of blueshifted one: ", np.argmin(all_z_obs))

print(min(all_z_obs), len(all_z_obs))

masked_all_z_obs = ma.masked_less_equal(all_z_obs, 0.0)
print(min(masked_all_z_obs), len(masked_all_z_obs))

np.count_nonzero(masked_all_z_obs.mask)


In [ ]:
small_gal_type = weights['Data/galaxy_type'][0:DATA_CUT_INDEX] # 0 1 2 3 possible
bins = plt.hist(small_gal_type, bins=50)

In [ ]:
small_z_obs = weights['Data/z_obs'][0:DATA_CUT_INDEX]
bins = plt.hist(small_z_obs, bins=50)


In [ ]:
print(min(small_z_obs), len(small_z_obs))
masked_small_z_obs = ma.masked_less_equal(small_z_obs, 0.00143)
print(min(masked_small_z_obs), len(masked_small_z_obs))

np.count_nonzero(masked_small_z_obs.mask)


In [ ]:
_cosmo = FlatLambdaCDM(H0=73, Om0=0.25, Ob0=0.045, Tcmb0=2.725, Neff=3.04) 
abs_mag = np.log10(_cosmo.luminosity_distance(-0.02).value)

vmax = get_max_observable_volume(abs_mag, -0.02, 19.5)
vmax

## Build a complete sky map of all the galaxies positions

In [ ]:
ra = weights['Data/ra'][:]#[0:DATA_CUT_INDEX]
dec = weights['Data/dec'][:]#[0:DATA_CUT_INDEX]

In [ ]:
ra_angles = coord.Angle(ra*u.degree)
ra_angles = ra_angles.wrap_at(180*u.degree)
dec_angles = coord.Angle(dec*u.degree)

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection="mollweide")
ax.scatter(ra_angles.radian, dec_angles.radian)
# This looks like Alex' paper, good
# TODO how to get frac_area from this?


## Examination of the MXXL HDF5 Data


In [ ]:
bins = plt.hist(weights['Data/abs_mag'][0:DATA_CUT_INDEX], bins=50)

In [ ]:
small_app_mag = weights['Data/app_mag'][0:DATA_CUT_INDEX]
bins = plt.hist(small_app_mag, bins=50)

In [ ]:
small_abs_mag = weights['Data/abs_mag'][0:DATA_CUT_INDEX]
bins = plt.hist(small_abs_mag, bins=50)

In [ ]:
my_abs_mag = app_mag_to_abs_mag(small_app_mag, small_z_obs)

In [ ]:
# TODO Compare my_abs_mag to abs_mag. 
# Calculating luminosity distances from the cosmology is slow
small_abs_mag

In [ ]:
# At what distance (luminosity distance) would the objects appear to be 19.5 mag?
v_max = get_max_observable_volume(my_abs_mag, small_z_obs, 19.5)

bins = plt.hist(np.log10(v_max), bins=50)
plt.xlabel("log(V_max) [Mpc]")
plt.ylabel("Count")

# Plots on data after running group finder

In [ ]:
def process(filename):

    filename_props = str.replace(filename, ".out", "_galprops.dat")

    df = pd.read_csv(filename, delimiter=' ', names=('RA', 'Dec', 'z', 'L_gal', 'V_max', 'P_sat', 'M_halo', 'N_sat', 'L_tot', 'igrp', 'unknown'))
    galprops = pd.read_csv(filename_props, delimiter=' ', names=('app_mag', 'g_r', 'galaxy_type', 'mxxl_halo_mass', 'fiber_assigned_0'))
    all_data = pd.merge(df, galprops, left_index=True, right_index=True)

    # Drop bad data, should have been cleaned up earlier though!
    orig_count = len(all_data)
    all_data = all_data[all_data.M_halo != 0]
    new_count = len(all_data)
    if (orig_count != new_count):
        print("Dropped {0} bad galaxies".format(orig_count - new_count))

    all_data['is_sat'] = (all_data.index != all_data.igrp).astype(int)
    #print(all_data['is_sat'])
    all_data['logLgal'] = np.log10(all_data.L_gal)

    bins = np.logspace(np.log10(min(all_data.M_halo)), np.log10(max(all_data.M_halo)), 30)
    labels = bins[0:len(bins)-1] # using bottom (or top?) value, not middle
    all_data['Mh_bin'] = pd.cut(x = all_data['M_halo'], bins = bins, labels = labels, include_lowest = True)
    
    centrals = all_data[all_data.index == all_data.igrp]
    #logmstar_means = centrals.groupby('Mh_bin').log_M_star.mean()
    #logmstar_scatter = centrals.groupby('Mh_bin').log_M_star.std()
    loglcen_means = centrals.groupby('Mh_bin').logLgal.mean()
    loglcen_scatter = centrals.groupby('Mh_bin').logLgal.std()

    # Compute f_sat(Lgal)
    bins_Lgal = np.logspace(np.log10(min(all_data.L_gal)), np.log10(max(all_data.L_gal)), 30)
    labels_Lgal = bins_Lgal[0:len(bins_Lgal)-1] # using bottom (or top?) value, not middle
    all_data['Lgal_bin'] = pd.cut(x = all_data['L_gal'], bins = bins_Lgal, labels = labels_Lgal, include_lowest = True)
    
    f_sat = all_data.groupby('Lgal_bin').is_sat.mean()
    Lgal_counts = all_data.groupby('Lgal_bin').RA.count()

    dataset = types.SimpleNamespace()
    dataset.all_data = all_data
    dataset.bins = bins
    dataset.labels = labels
    dataset.centrals = centrals
    #dataset.logmstar_means = logmstar_means
    #dataset.logmstar_scatter = logmstar_scatter
    dataset.loglcen_means = loglcen_means
    dataset.loglcen_scatter = loglcen_scatter
    dataset.bins_Lgal = bins_Lgal
    dataset.labels_Lgal = labels_Lgal
    dataset.f_sat = f_sat
    dataset.Lgal_counts = Lgal_counts

    return dataset

def plots(*frames):
    
    plt.figure()
    for f in frames:
        plt.errorbar(f.labels, f.loglcen_means, yerr=f.loglcen_scatter)
    plt.xscale('log')
    plt.xlabel('$M_{halo} / h$')
    plt.ylabel('$log(L_{cen})$')
    plt.title("Central Luminosity vs. Halo Mass")
    plt.legend()
    plt.draw()

    plt.figure()    
    for f in frames:
        plt.errorbar(f.labels, f.loglcen_scatter)
    plt.xscale('log')
    plt.xlabel('$M_{halo} / h$')
    plt.ylabel('$\\sigma(\\log(L_{cen})$')
    plt.title("Central Luminosity Scatter vs. Halo Mass")
    plt.draw()

    """     
    plt.figure()
    for f in frames:
        plt.scatter(f.centrals.M_halo, f.centrals.L_gal, alpha=0.002)
    plt.loglog()
    plt.xlabel('M_halo / h')
    plt.ylabel('L_gal / $h^2$)')
    plt.draw() 
    """

    plt.figure()
    for f in frames:
        plt.hist(f.centrals.N_sat, np.arange(0,50,1), alpha=0.5)
    plt.yscale('log')
    plt.ylabel("Number of Satellites")
    plt.xlabel("Count of Centrals")
    plt.title("How many satellites do Centrals have?")
    plt.draw()

    plt.figure()
    for f in frames:
        plt.plot(f.labels_Lgal, f.f_sat)
    plt.xscale('log')
    plt.xlabel("$L_{gal}$")
    plt.ylabel("$f_{sat}$")
    plt.title("Satellite fraction vs Galaxy Luminosity")
    plt.draw()
    
    plt.figure()
    for f in frames:
        plt.plot(f.labels_Lgal, f.Lgal_counts)
    plt.xscale('log')
    plt.yscale('log')
    plt.xlabel("$L_{gal}$")
    plt.ylabel("Count of Galaxies")
    plt.title("Galaxy Luminosity Counts")
    plt.draw()    

In [ ]:
mini = process(ROOT_FOLDER + "mxxl_3pass_all_mini.out")
plots(mini)


In [ ]:
all = process(ROOT_FOLDER + "mxxl_3pass_all.out")


In [ ]:
fiberonly = process(ROOT_FOLDER + "mxxl_3pass_fiberonly.out")


In [ ]:
nn = process(ROOT_FOLDER + "mxxl_3pass_nn.out")

In [ ]:
plots(all, fiberonly, nn)